In [22]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam

model = Sequential()
model.add(Conv2D(16, (5, 5), padding='same', input_shape=(48, 48, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (5, 5), padding='same', ))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (5, 5), padding='same', ))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation="softmax"))
# model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 48, 48, 16)        1216      
_________________________________________________________________
activation_17 (Activation)   (None, 48, 48, 16)        0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 24, 24, 32)        12832     
_________________________________________________________________
activation_18 (Activation)   (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 12, 12, 32)       

In [24]:
batch_size = 16

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   # rotation_range=10,
                                   # width_shift_range=0.2,
                                   # height_shift_range=0.2,
                                   # horizontal_flip=True,
                                   validation_split=0.2)

val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = train_datagen.flow_from_directory('./Datawhale_人脸情绪识别_数据集/train/', 
                                               color_mode='rgb',
                                               batch_size=batch_size, 
                                               class_mode='categorical',
                                               target_size=(48, 48),
                                               shuffle=True,
                                               subset = 'training') 

val_data = val_datagen.flow_from_directory('./Datawhale_人脸情绪识别_数据集/train/', 
                                           color_mode='rgb',
                                           batch_size=batch_size, 
                                           class_mode='categorical',
                                           target_size=(48, 48),
                                           shuffle=False,
                                           subset = 'validation')

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.


In [25]:
model.fit_generator(
        train_data,
        epochs=5,
        validation_data=val_data,
        validation_steps=200)
model.save_weights('first_try.h5') 

Epoch 1/5
1436/1436 [==============================] - 6s 4ms/step - loss: 1.8434 - accuracy: 0.2410 - val_loss: 1.6486 - val_accuracy: 0.4384
Epoch 2/5
1436/1436 [==============================] - 5s 4ms/step - loss: 1.6569 - accuracy: 0.3452 - val_loss: 1.4604 - val_accuracy: 0.4281
Epoch 3/5
1436/1436 [==============================] - 5s 4ms/step - loss: 1.5086 - accuracy: 0.4159 - val_loss: 1.3834 - val_accuracy: 0.4753
Epoch 4/5
1436/1436 [==============================] - 5s 4ms/step - loss: 1.4250 - accuracy: 0.4439 - val_loss: 1.3349 - val_accuracy: 0.4853
Epoch 5/5
1436/1436 [==============================] - 5s 4ms/step - loss: 1.3625 - accuracy: 0.4729 - val_loss: 1.2874 - val_accuracy: 0.4822


In [37]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        './Datawhale_人脸情绪识别_数据集/',
        target_size=(48, 48),
        color_mode="rgb",
        shuffle = False,
        class_mode='categorical',
        batch_size=10)

filenames = test_generator.filenames
nb_samples = len(filenames)
predict = model.predict_generator(test_generator,steps = nb_samples)

Found 35887 images belonging to 2 classes.


In [51]:
train_data.class_indices

{'angry': 0,
 'disgusted': 1,
 'fearful': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprised': 6}

In [59]:
import numpy as np
import pandas as pd
cls_name = np.array(['angry', 'disgusted', 'fearful', 'happy','neutral', 'sad', 'surprised'])

In [61]:
submit_df = pd.DataFrame({'name': filenames, 'label': cls_name[predict.argmax(1)]})

In [62]:
submit_df.head()

,name,label
0,test/00001.png,fearful
1,test/00002.png,sad
2,test/00003.png,happy
3,test/00004.png,fearful
4,test/00005.png,happy


In [69]:
submit_df = submit_df[submit_df['name'].apply(lambda x: 'test' in x )]
submit_df['name'] = submit_df['name'].apply(lambda x: x.split('/')[-1])

In [72]:
submit_df = submit_df.sort_values(by='name')
submit_df.to_csv('submit.csv', index=None)